  # Coursera Capstone Project
<b>This is the capstone project in the IBM skillshare network course</b>

# FINDING THE SAFEST PLACE IN SAN FRANCISCO TO LIVE 

<h3>Importing libraries

In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests


<h3>Installing and importing folium

In [2]:
import folium

<h2>Downloading San Fransico crime dataset that we used in data visualization course

In [3]:
san_crime = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DV0101EN/labs/Data_Files/Police_Department_Incidents_-_Previous_Year__2016_.csv')


In [4]:
san_crime.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


<strong>Limiting to the first 1000 cases 

In [5]:
limit = 1000
san_crime = san_crime.iloc[0:limit, :]

In [6]:
san_crime.shape

(1000, 13)

<h3> Finding the coordinates of San Fancisco

In [7]:
address = 'San Francisco'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


<h3>Plotting the crimes and grouping them into clusters

In [8]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
sanfran_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(sanfran_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(san_crime.Y, san_crime.X, san_crime.Category):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
sanfran_map

<strong>From the map, we can see that the south-west part of the city has the lowest amount of crimes, but upon closer inspection using the GROUPBY method on 'PdDistrict', we can see that Richmond areas has the lowest crime rate (we aviod the cluster with value 2 because of less area)

In [9]:
san_crime.groupby(['PdDistrict']).count()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,Resolution,Address,X,Y,Location,PdId
PdDistrict,,,,,,,,,,,,
BAYVIEW,106,106,106,106,106,106,106,106,106,106,106,106
CENTRAL,108,108,108,108,108,108,108,108,108,108,108,108
INGLESIDE,99,99,99,99,99,99,99,99,99,99,99,99
MISSION,146,146,146,146,146,146,146,146,146,146,146,146
NORTHERN,103,103,103,103,103,103,103,103,103,103,103,103
PARK,67,67,67,67,67,67,67,67,67,67,67,67
RICHMOND,53,53,53,53,53,53,53,53,53,53,53,53
SOUTHERN,166,166,166,166,166,166,166,166,166,166,166,166
TARAVAL,68,68,68,68,68,68,68,68,68,68,68,68


<h3> Getting the location coordinates of Richmond Area

In [10]:
address = 'RICHMOND, San Francisco'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Richmond,San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Richmond,San Francisco are 37.7770459, -122.4654532.


<h3>Selecting crime reports of only Richmond area

In [11]:
san_crime_rich = san_crime.loc[san_crime["PdDistrict"]=='RICHMOND']

In [12]:
san_crime_rich.shape

(53, 13)

In [13]:
san_crime_rich.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
24,160006071,ASSAULT,THREATS AGAINST LIFE,Sunday,01/03/2016 12:00:00 AM,09:38,RICHMOND,"ARREST, BOOKED",FULTON ST / 47TH AV,-122.507750,37.771494,"(37.7714939969416, -122.507750131004)",16000607119057
93,160014397,OTHER OFFENSES,FALSE PERSONATION,Monday,01/04/2016 12:00:00 AM,15:10,RICHMOND,NONE,700 Block of 40TH AV,-122.500380,37.774621,"(37.7746206491065, -122.500380427915)",16001439709027
94,160014836,ROBBERY,ROBBERY OF A CHAIN STORE WITH A KNIFE,Wednesday,01/06/2016 12:00:00 AM,14:23,RICHMOND,NONE,700 Block of CABRILLO ST,-122.466559,37.775192,"(37.7751918267217, -122.466558780683)",16001483603042
146,160018474,NON-CRIMINAL,FOUND PROPERTY,Thursday,01/07/2016 12:00:00 AM,10:00,RICHMOND,NONE,300 Block of 14TH AV,-122.472982,37.781571,"(37.7815707596523, -122.472981975493)",16001847472000
166,160219036,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Tuesday,03/15/2016 12:00:00 AM,16:20,RICHMOND,NONE,ANZA ST / BEAUMONT AV,-122.454819,37.780116,"(37.7801162668233, -122.454819341663)",16021903606244


<h3>Plotting the crimes of Richmond Area to find the area with minimum recorded crimes

In [14]:
sanfran_map_rich = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(sanfran_map_rich)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(san_crime_rich.Y, san_crime_rich.X, san_crime_rich.Category):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
sanfran_map_rich

<h3>We see upon zooming into the 5-counted cluster that the location name is California Street, and now we retrieve the coordinates

In [15]:
address = 'California Street, San Francisco'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of California Street,San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of California Street,San Francisco are 37.7872487, -122.4469132.


<h2> Using the Foursquare API to get the 20 most popular places

In [16]:
# The code was removed by Watson Studio for sharing.

In [17]:
radius = 500
LIMIT = 20
venues = []
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        latitude,
        longitude,
        radius, 
        LIMIT)
results = requests.get(url).json()["response"]['groups'][0]['items']
for venue in results:
        venues.append((
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [19]:
venues_df = pd.DataFrame(venues)
venues_df

,0,1,2,3
0,Pet Food Express,37.787406,-122.446326,Pet Store
1,JCC Fitness Center,37.787215,-122.447671,Gym
2,Garibaldi's,37.788387,-122.447130,American Restaurant
3,Sorrel,37.788426,-122.446187,New American Restaurant
4,Vogue Theater,37.788396,-122.446773,Indie Movie Theater
5,Bryan's Market,37.786517,-122.450937,Gourmet Shop
6,Rendez-Vous Nails,37.788294,-122.446438,Spa
7,A Runners Mind - San Francisco,37.787595,-122.451473,Sporting Goods Shop
8,March,37.788423,-122.443473,Furniture / Home Store
9,Entangle & Sway Pole Dance Fitness,37.785754,-122.443224,Gym / Fitness Center


<h2>Upon inspecting the locations returned by the Foursquare API, we can assume that it is a good place to start a business / to live as most of the necessary businesses like spa, gym, stores, restuarents etc, are nearby